In [37]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Preliminary functions
Functions that make our life easier, since we will be reusing some code.

In [ ]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. 
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [30]:
def onv_bases(N, K, transition = "detachment"):
    """
    Create two ONV bases, based on whether we're describing an electron attachment or detachment.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    if transition == "detachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha - 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    elif transition == "attachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    else:
        print("Specify a valid electron transition process: attachment/detachment.")
        return None
    
    return (onv_basis_I, onv_basis_J)

In [2]:
def UHF(molecule):
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    print(molecule)
    print("N_alpha: {}\nN_beta: {}\nK: {}\n----------------------------------------\n".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    
    return uhf_parameters.expansion()

In [3]:
def uFCI(molecule, onv_basis):
    u_spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    u_spin_orbital_basis.lowdinOrthonormalize()
    
    u_hamiltonian = gqcpy.USQHamiltonian.Molecular(u_spin_orbital_basis, molecule)
    
    solver = gqcpy.EigenproblemSolver.Dense()
    environment = gqcpy.CIEnvironment.Dense(u_hamiltonian, onv_basis)

    ci_parameters = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
    
    return ci_parameters

# Preparation of some variables

Generate the neutral molecule with N electrons, a positive ion with N-1 electrons (charge +1) and a negative ion with N+1 electrons (charge -1).

In [4]:
neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule with N electrons
positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion N-1 electrons
negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with N+1 electrons

Build an ONV basis for a specified electron transition process, given the dimensions N and K. Here, we choose an electron detachment.

In [31]:
N, M = dimensions(neutral_molecule)
K = M//2 # number of spinors per spin compound

onv_basis_I, onv_basis_J = onv_bases(N, K, "detachment")

# Dyson orbitals in the uncorrelated limit

Build the linear expansion for a CI wave function in the Hartree-Fock formalism, where all coefficients of a given ONV basis are zero except for the HF determinant.

In [26]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_I)
Psi_J = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_J)

Construct Dyson orbitals using the "pseudo overlap" between an N-1 wave function and a N wave function with an annihilated electron in each spinor.

In [27]:
d_IJ = gqcpy.DysonOrbital.TransitionAmplitudes(Psi_J, Psi_I)
d_IJ.amplitudes()

array([0., 0., 0., 0., 1., 0., 0.])

In a Hartree-Fock picture, the difference of two electron densities, one of an $N$-electron system and the other of a $(N-1)$-electron system, should be reduced to the norm of a Dyson orbital. For a more profound theoretical explanation, the reader is referred to [this page](https://gqcg-res.github.io/dariatols_knowdes/correlation-in-the-dyson-orbital-picture.html#uncorrelated-limit). 

In a spin-orbital basis, we have an $\alpha$ and $\beta$ 1-DM and the total 1-DM is the sum of both parts.

In [53]:
electron_density_I = np.sum( np.diagonal( Psi_I.calculate1DM() ) )
electron_density_J = np.sum( np.diagonal( Psi_J.calculate1DM() ) )

print("Electron density for |N-1, (I)>:")
print(electron_density_I)

print("\n" + "-"*25 + "\n")

print("Electron density for |N, (J)>:")
print(electron_density_J)

Electron density for |N-1, (I)>:
9.0

-------------------------

Electron density for |N, (J)>:
10.0


In [55]:
print("Is the difference between the N and (N-1) electron density equal to the Dyson orbital that represents a transition amplit")
print( electron_density_J - electron_density_I == np.sum(d_IJ.amplitudes()) )

True

True

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.